In [151]:
import numpy as np

D = 784
L = 12
a = np.random.uniform(-1, 1, D)
b = np.random.uniform(-1, 1, L)
x = np.random.randint(0, 2, D)
w = np.random.uniform(-1, 1, (D, L))


# TODO: implement sum of G(z) over z in Num and Den
class LOGlike(object):
    """
    FORMAT:
    D = 784   # visible
    L = 12    # hidden
    a = np.random.uniform(0, 1, D)
    b = np.random.uniform(0, 1, L)
    x = np.random.randint(0, 2, D)
    z = np.random.randint(0, 2, L)
    w = np.random.uniform(0, 1, (D, L))
    """
    def make_z_states(self):
        def get_bin(i, L):
            b = bin(i)[2:]
            return ("0" * (L - len(b))) + b
            #
        #####
        #
        self.zs = np.array([[int(bit) for bit in get_bin(i, self.L)] for i in range(2 ** self.L)])
        #
    #####
    #   
    def __init__(self, a, b, x, w):
        self.a = a   # visible
        self.b = b   # hidden
        self.D = a.shape[0]
        self.L = b.shape[0]
        self.x = x
        self.w = w
        self.q = None
        self.sum  = lambda x: np.array(x).sum()
        self.prod = lambda x: np.array(x).prod()
        self.mean = lambda x: np.array(x).mean()
        self.H    = lambda i, z: self.a[i] + self.sum([self.w[i][mu] * self.zs[z][mu] for mu in range(self.L)])
        self.G    = lambda z:    self.prod([np.exp(self.b[mu] * self.zs[z][mu])       for mu in range(self.L)])
        self.esit = None
        self.zs   = None
        self.make_z_states()
        #
    #####
    #
    def run(self):
        
        def get_numerator():
            numerator = self.sum([self.G(z) * self.prod([np.exp(x[i] * self.H(i, z))
                                                            for i in range(self.D)])
                                                            for z in range(self.L)])
            return np.log(numerator)
            #
        #####
        #
        def get_denominator():
            self.q = np.array([self.mean([self.H(i, z) for i in range(self.D)]) for z in range(self.L)])
            self.q = (1 + np.exp(self.q)).mean()
            esit   = self.sum([self.G(z) * self.prod([(1 + np.exp(self.H(i, z))) / self.q
                                                                 for i in range(self.D)])
                                                                 for z in range(self.L)])
            esit   = np.log(esit)
            return - self.D * self.q - np.log(self.q)
            #
        #####
        #
        self.esit = get_numerator() + get_denominator()
        #
    #####
    #
    def print(self):
        return self.esit
        #
    #####
    #
#####

log_likehood0 = LOGlike(a, b, x, w)
log_likehood0.run()
log_likehood0.print()


-1523.025914090904